In [17]:
from IPython.display import display

from backtesting import Backtest, Strategy
import yfinance as yf

In [18]:

class FallUp(Strategy):
	def init(self):
		self.down_days = 0
		self.hold_days = 0

	def next(self):
		# close price of previous day
		pv = self.data.Close[-2]

		# close price of current day
		cv = self.data.Close[-1]

		# consecutively falling 3 times and reversal, buy-in
		if self.down_days >= 3 and cv > pv:
			self.buy()
			self.hold_days = 0
			self.down_days_after_position = 0

		# record falling times
		if cv < pv:
			self.down_days += 1
		else:
			self.down_days = 0

		if self.position:
			self.hold_days += 1

			# record falling times after position
			if cv < pv:
				self.down_days_after_position += 1

			# in any window, falling over 2 times, close the position
			# backtesting can do long / short trading, self.sell() means shorting, 
			# close position should use self.position.close()
			if self.down_days_after_position > 1:
				self.position.close()
			
			# with position, falling once within 4 times, close the position
			if self.hold_days <= 4 and self.down_days_after_position > 0:
				self.position.close()

In [19]:

bt = Backtest(
	# data
	yf.download("^SPX", period="60d", interval="5m"), 

	# Strategy
	FallUp, 

	# Commission
	commission=.002, 

	# market orders will be filled with respect to the current bar's closing price instead of the next bar's open.
	trade_on_close=True,

	# new order auto-closes the previous trade/position, making at most a single trade (long or short) in effect at each time
	exclusive_orders=True 
)


[*********************100%%**********************]  1 of 1 completed


In [20]:
stats = bt.run()

In [21]:
display(stats)

Start                     2023-11-27 09:30...
End                       2024-02-16 15:55...
Duration                     81 days 06:25:00
Exposure Time [%]                   16.644175
Equity Final [$]                  7326.443839
Equity Peak [$]                       10000.0
Return [%]                         -26.735562
Buy & Hold Return [%]               10.020946
Return (Ann.) [%]                  -73.799219
Volatility (Ann.) [%]                 1.05371
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -26.735562
Avg. Drawdown [%]                  -26.735562
Max. Drawdown Duration       81 days 05:30:00
Avg. Drawdown Duration       81 days 05:30:00
# Trades                                  235
Win Rate [%]                         2.978723
Best Trade [%]                       0.130934
Worst Trade [%]                     -0.805241
Avg. Trade [%]                    

In [22]:
display(stats._trades)

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2,11,13,4561.083940,4554.029785,-14.108311,-0.001547,2023-11-27 10:25:00-05:00,2023-11-27 10:35:00-05:00,0 days 00:10:00
1,2,35,37,4564.450523,4555.250000,-18.401047,-0.002016,2023-11-27 12:25:00-05:00,2023-11-27 12:35:00-05:00,0 days 00:10:00
2,2,53,55,4565.873285,4556.830078,-18.086414,-0.001981,2023-11-27 13:55:00-05:00,2023-11-27 14:05:00-05:00,0 days 00:10:00
3,2,64,66,4558.338715,4551.899902,-12.877625,-0.001413,2023-11-27 14:50:00-05:00,2023-11-27 15:00:00-05:00,0 days 00:10:00
4,2,69,75,4562.266477,4553.470215,-17.592523,-0.001928,2023-11-27 15:15:00-05:00,2023-11-27 15:45:00-05:00,0 days 00:30:00
...,...,...,...,...,...,...,...,...,...,...
230,1,4402,4404,5041.302715,5031.540039,-9.762676,-0.001937,2024-02-16 12:20:00-05:00,2024-02-16 12:30:00-05:00,0 days 00:10:00
231,1,4407,4408,5041.252321,5030.100098,-11.152224,-0.002212,2024-02-16 12:45:00-05:00,2024-02-16 12:50:00-05:00,0 days 00:05:00
232,1,4413,4415,5031.012155,5026.799805,-4.212351,-0.000837,2024-02-16 13:15:00-05:00,2024-02-16 13:25:00-05:00,0 days 00:10:00
233,1,4433,4434,5025.741381,5013.629883,-12.111498,-0.002410,2024-02-16 14:55:00-05:00,2024-02-16 15:00:00-05:00,0 days 00:05:00


In [23]:
bt.plot()

/Users/kimfaicheang/workspace/snowball/venv/lib/python3.10/site-packages/backtesting/_plotting.py:455: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


Row(id='2788', ...)